In [ ]:
from tqdm import tqdm
import random
import re
import os
from google.colab import drive
import json
import pandas as pd

drive.mount('/content/drive')

folder_path = '/content/drive/My Drive/HSE/programming/DL in NLP/Project'

for file in os.listdir(folder_path):
    print(file)

Mounted at /content/drive
synonims.csv
RuDReC NER.gslides
clinical_corpus
BIO_data.csv
annotation
sentence_data_splitted
label2id.json
id2label.json
model
augmentations
ner_sent_results
models_ner_sent
NER+sent_classification.ipynb
models_ner
NER.ipynb
errors_analysis.ipynb
RuDrec_project.gdoc


In [ ]:
!wget "https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json"

--2024-03-24 17:55:00--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json’

rudrec_annotated.js 100%[===================>]   1.69M  9.38MB/s    in 0.2s    

2024-03-24 17:55:01 (9.38 MB/s) - ‘rudrec_annotated.json’ saved [1773014/1773014]



In [ ]:
datapath = './rudrec_annotated.json'

all_lines = []
with open(datapath) as f:
    for line in f:
        file_ = json.loads(line)
        all_lines.append(file_)

In [ ]:
data = pd.DataFrame(all_lines)
data.head()

,file_name,text,entities,sentence_id
0,172744.tsv,"нам прописали, так мой ребенок сыпью покрылся,...","[{'start': 122, 'entity_type': 'Drugform', 'en...",0
1,172744.tsv,Общее впечатление : не подошел\n,[],1
2,592814.tsv,Пила этот препарат для повышения иммунитета 5 ...,"[{'start': 23, 'entity_type': 'DI', 'end': 43,...",0
3,592814.tsv,"Так как начала работать в аптеке, начала часто...",[],1
4,592814.tsv,В месяц по нескольку раз причем со всеми вытек...,"[{'start': 66, 'entity_type': 'DI', 'end': 72,...",2


## Синонимы

In [ ]:
synonyms_df = pd.read_csv(folder_path + '/synonims.csv', sep='\t', index_col=0)
synonyms_df.head()

,concept_id,entity_text,entity_type
0,C0000731,в животе было вздутие,ADR
1,C0000731,вздутие живота,ADR
2,C0001047,ацц,Drugname
3,C0001367,ацикловир сандоз,Drugname
4,C0001367,ацикловира,Drugname


In [ ]:
def augment_text_and_update_indices(df, synonyms_df):
    augmented_data = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        text = row['text']
        updated_entities = []
        offset = 0  # Смещение, вызванное заменой текста

        for entity in row['entities']:
            if 'concept_id' in entity:
                concept_id = entity['concept_id']
                synonyms = synonyms_df[synonyms_df['concept_id'] == concept_id]['entity_text'].tolist()
                if synonyms:
                    synonym = random.choice(synonyms)
                    original_text_length = entity['end'] - entity['start']
                    new_text_length = len(synonym)
                    text = text[:entity['start'] + offset] + synonym + text[entity['end'] + offset:]
                    updated_entities.append({
                        "start": entity['start'] + offset,
                        "entity_type": entity['entity_type'],
                        "end": entity['start'] + offset + new_text_length,
                        "entity_id": entity['entity_id'],
                        "entity_text": synonym,
                        "concept_id": entity['concept_id'],
                        "concept_name": entity.get('concept_name')
                    })
                    offset += new_text_length - original_text_length  # Обновляем смещение для следующих сущностей

        augmented_data.append({
            "file_name": row['file_name'],
            "text": text,
            "entities": updated_entities,
            "sentence_id": row['sentence_id']
        })
    return augmented_data

In [ ]:
augmented_data = augment_text_and_update_indices(data, synonyms_df)

100%|██████████| 4809/4809 [00:02<00:00, 1604.78it/s]


In [ ]:
augmented_data[:3]

[{'file_name': '172744.tsv',
  'text': 'нам прописали, так мой ребенок сыпь уже была на груди, глаза опухли, сверху и снизу высыпания, ( 8 месяцев сыну)А от виферону такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
  'entities': [{'start': 122,
    'entity_type': 'Drugform',
    'end': 130,
    'entity_id': '*[0]_se',
    'entity_text': 'виферону',
    'concept_id': 'C0021735',
    'concept_name': nan},
   {'start': 31,
    'entity_type': 'ADR',
    'end': 53,
    'entity_id': '*[1]',
    'entity_text': 'сыпь уже была на груди',
    'concept_id': 'C0015230',
    'concept_name': nan},
   {'start': 55,
    'entity_type': 'ADR',
    'end': 67,
    'entity_id': '*[2]',
    'entity_text': 'глаза опухли',
    'concept_id': 'C4760994',
    'concept_name': nan},
   {'start': 84,
    'entity_type': 'ADR',
    'end': 93,
    'entity_id': '*[3]',
    'entity_text': 'высыпания',
    'concept_id': 'C0015230',
    'concept_name': nan}],
  'sentence_id': 0},
 {'file_name': 

In [ ]:
file_path = 'augmented_synonyms.json'

with open(file_path, 'w', encoding='utf-8') as f:
    for item in augmented_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
umls_df = pd.read_csv('umls_concepts.csv')
augmented_data = augment_text_and_update_indices(data, umls_df)

100%|██████████| 4809/4809 [00:03<00:00, 1418.80it/s]


In [ ]:
augmented_data[:3]

[{'file_name': '172744.tsv',
  'text': 'нам прописали, так мой ребенок экзантема, опухшие глаза, сверху и снизу экзантема, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
  'entities': [{'start': 31,
    'entity_type': 'ADR',
    'end': 40,
    'entity_id': '*[1]',
    'entity_text': 'экзантема',
    'concept_id': 'C0015230',
    'concept_name': nan},
   {'start': 42,
    'entity_type': 'ADR',
    'end': 55,
    'entity_id': '*[2]',
    'entity_text': 'опухшие глаза',
    'concept_id': 'C4760994',
    'concept_name': nan},
   {'start': 72,
    'entity_type': 'ADR',
    'end': 81,
    'entity_id': '*[3]',
    'entity_text': 'экзантема',
    'concept_id': 'C0015230',
    'concept_name': nan}],
  'sentence_id': 0},
 {'file_name': '172744.tsv',
  'text': 'Общее впечатление : не подошел\n',
  'entities': [],
  'sentence_id': 1},
 {'file_name': '592814.tsv',
  'text': 'Пила этот препарат для бустерная доза вакцины 5 лет назад.\n',
  

In [ ]:
file_path = 'augmented_umls.json'

with open(file_path, 'w', encoding='utf-8') as f:
    for item in augmented_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

## UMLS

In [ ]:
!pip install transliterate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.4 MB/s eta 0:00:00


In [ ]:
import re
from transliterate import translit
import requests

def improve_transliteration(text):
    text = text.replace('тс', 'ц')
    text = text.replace('цх', 'ч')
    text = text.replace('иа', 'я')
    return text

def is_cyrillic(text):
    return bool(re.search('[\u0400-\u04FF]', text))

def ensure_cyrillic(name):
    if is_cyrillic(name):
        return name.lower()
    else:
        return improve_transliteration(translit(name, 'ru').lower())

def get_names_for_concept(apikey, version, concept_id):
    names = []
    content_endpoint = f'https://uts-ws.nlm.nih.gov/rest/content/{version}/CUI/{concept_id}'
    query = {'apiKey': apikey, 'language': 'RUS'}
    response = requests.get(content_endpoint, params=query)
    response.encoding = 'utf-8'

    # Проверка статуса ответа и вывод сообщения в случае ошибки
    if response.status_code != 200:
        print(f"Concept ID '{concept_id}' not found in UMLS.")
        return names  # Возвращаем пустой список, если концепт не найден

    items = response.json()
    jsonData = items.get('result', {})

    # Проверка наличия атомов в ответе
    if 'atoms' in jsonData and jsonData['atoms']:
        Atoms = jsonData['atoms']
        page = 0

        try:
            while True:
                page += 1
                atom_query = {'apiKey': apikey, 'pageNumber': page}
                a = requests.get(Atoms, params=atom_query)
                a.encoding = 'utf-8'

                if a.status_code != 200:
                    break

                all_atoms = a.json()
                jsonAtoms = all_atoms.get('result', [])

                for atom in jsonAtoms:
                    if atom['language'] == 'RUS':
                        name = ensure_cyrillic(atom['name'])
                        if name not in names:  # Проверка на уникальность
                            names.append(name)
        except Exception as except_error:
            print(f"Error processing concept ID '{concept_id}': {except_error}")
    else:
        print(f"No Russian atoms found for concept ID '{concept_id}'.")

    return names

In [ ]:
apikey = 'a7dc2f34-3b9f-440e-9ecf-08dce62cc067'
version = 'current'
identifier = 'C0020971'  # Пример CUI
get_names_for_concept(apikey, version, identifier)

['иммунизация',
 'вакцинация',
 'иммунологическая стимуляция',
 'иммуностимуляция',
 'сенсибилизация иммунологическая',
 'иммунизации',
 'бустерная доза вакцины']

In [ ]:
# Извлечение уникальных concept_id
unique_concept_ids = set()
for row in data['entities']:
    for entity in row:
        if 'concept_id' in entity:
            unique_concept_ids.add(entity['concept_id'])

In [ ]:
import pandas as pd
from tqdm import tqdm

rows = []
for concept_id in tqdm(unique_concept_ids):
    names = get_names_for_concept(apikey, version, concept_id)
    for name in names:
        rows.append({'concept_id': concept_id, 'entity_text': name.lower()})

# Удаление дубликатов
unique_rows = [dict(t) for t in {tuple(d.items()) for d in rows}]

new_df = pd.DataFrame(unique_rows, columns=['concept_id', 'entity_text'])

  4%|▍         | 21/530 [01:12<24:22,  2.87s/it]

Concept ID 'C0010200C0010200' not found in UMLS.


  8%|▊         | 41/530 [02:19<26:44,  3.28s/it]

Concept ID 'None' not found in UMLS.


 10%|█         | 54/530 [03:01<25:14,  3.18s/it]

...

 95%|█████████▌| 505/530 [25:49<00:57,  2.28s/it]

Concept ID 'C1272745|C0037313' not found in UMLS.


 99%|█████████▉| 527/530 [26:59<00:06,  2.17s/it]

Concept ID 'C0003419|C3178748' not found in UMLS.


100%|██████████| 530/530 [27:10<00:00,  3.08s/it]


In [ ]:
new_df.head()

,concept_id,entity_text
0,C0043352,сухая ротовая полость
1,C3887612,беспокойство
2,C0021051,иммунологической недостаточности синдромы
3,C0025517,нарушение обмена веществ
4,C0020281,перекись водорода


In [ ]:
len(new_df)

1579

In [ ]:
new_df.to_csv('umls_concepts.csv', index=False, encoding='utf-8')

In [ ]:
! pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 40.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f881c6934cac954d60d75185cb413ed813f461d7ae082cc186e5f53fe8eb85b4
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 911.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 10.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pymorphy2

nlp = spacy.load('ru_core_news_sm')
morph = pymorphy2.MorphAnalyzer()

def match_case_and_number(original_phrase, new_phrase):
    doc_original = nlp(original_phrase)
    main_word_token = None

    # Поиск главного существительного в исходном словосочетании
    for token in doc_original:
        if token.pos_ == 'NOUN':
            main_word_token = token
            break

    # Если главное существительное не найдено, возвращаем новое словосочетание
    if not main_word_token:
        return new_phrase

    parsed_main_word = morph.parse(main_word_token.text)[0]
    case = parsed_main_word.tag.case
    number = parsed_main_word.tag.number

    doc_new = nlp(new_phrase)
    inflected_phrase = []

    for token in doc_new:
        parsed_new_word = morph.parse(token.text)[0]
        if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
            # Инфлектируем только если это главное существительное
            if token.dep_ == 'ROOT' or token.head.pos_ != 'NOUN' or token.pos_ == 'ADJ':
                if case and number:
                    inflected_params = {case, number} - {None}
                    inflected_word = parsed_new_word.inflect(inflected_params)
                    inflected_phrase.append(inflected_word.word if inflected_word else token.text)
                else:
                    inflected_phrase.append(token.text)
            else:
                inflected_phrase.append(token.text)
        else:
            inflected_phrase.append(token.text)

    return ' '.join(inflected_phrase)

result = match_case_and_number('реакция аллергическая', 'снижением иммунитета')
print(result)

снижение иммунитета


In [ ]:
match_case_and_number('головной болью', 'сильная мигрень')

'сильной мигренью'

In [ ]:
match_case_and_number('снижением иммунитета', 'аллергия')

'аллергией'

### Замена с инфлектированием по числу и падежу

In [ ]:
def augment_text_and_update_indices(df, synonyms_df):
    augmented_data = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        text = row['text']
        updated_entities = []
        offset = 0

        for entity in row['entities']:
            if 'concept_id' in entity:
                concept_id = entity['concept_id']
                synonyms = synonyms_df[synonyms_df['concept_id'] == concept_id]['entity_text'].tolist()
                if synonyms:
                    synonym = random.choice(synonyms)
                    original_word = text[entity['start'] + offset : entity['end'] + offset]
                    matched_synonym = match_case_and_number(original_word, synonym)
                    new_text_length = len(matched_synonym)
                    text = text[:entity['start'] + offset] + matched_synonym + text[entity['end'] + offset:]
                    updated_entities.append({
                        "start": entity['start'] + offset,
                        "entity_type": entity['entity_type'],
                        "end": entity['start'] + offset + new_text_length,
                        "entity_id": entity['entity_id'],
                        "entity_text": matched_synonym,
                        "concept_id": entity['concept_id'],
                        "concept_name": entity.get('concept_name')
                    })
                    offset += new_text_length - (entity['end'] - entity['start'])

        augmented_data.append({
            "file_name": row['file_name'],
            "text": text,
            "entities": updated_entities,
            "sentence_id": row['sentence_id']
        })
    return augmented_data

In [ ]:
umls_df = pd.read_csv('umls_concepts.csv')

In [ ]:
augmented_data = augment_text_and_update_indices(data, umls_df)

100%|██████████| 4809/4809 [00:54<00:00, 87.93it/s] 


In [ ]:
file_path = 'augmented_umls.json'

with open(file_path, 'w', encoding='utf-8') as f:
    for item in augmented_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
augmented_data = augment_text_and_update_indices(data, synonyms_df)

100%|██████████| 4809/4809 [01:02<00:00, 77.31it/s] 


In [ ]:
file_path = 'augmented_synonyms.json'

with open(file_path, 'w', encoding='utf-8') as f:
    for item in augmented_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

## BERT

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer
import random
import re

model_name = "DeepPavlov/rubert-base-cased"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

def augment_text_and_entities(text, entities):
    augmented_entities = []
    text_tokens = re.findall(r'\b\w+\b', text, flags=re.UNICODE)

    if random.choice([True, False]) and entities:
        entity = random.choice(entities)
        start, end = entity['start'], entity['end']
        masked_text = text[:start] + tokenizer.mask_token + text[end:]
        outputs = fill_mask(masked_text)

        if outputs:
            for output in outputs:
                if output['sequence'].strip() != text.strip():
                    new_entity_text = output['token_str']
                    delta = len(new_entity_text) - (end - start)
                    text = text[:start] + new_entity_text + text[end:]
                    for ent in entities:
                        if ent['start'] > end:
                            ent['start'] += delta
                            ent['end'] += delta
                    entity['end'] = start + len(new_entity_text)
                    entity['entity_text'] = new_entity_text
                    break
        augmented_entities.extend([dict(ent) for ent in entities])
    else:
        non_entity_word_indexes = [i for i, token in enumerate(text_tokens) if all(i < ent['start'] or i > ent['end'] for ent in entities)]
        if non_entity_word_indexes:
            replace_index = random.choice(non_entity_word_indexes)
            word_start = text.find(text_tokens[replace_index])
            word_end = word_start + len(text_tokens[replace_index])
            masked_text = text[:word_start] + tokenizer.mask_token + text[word_end:]
            outputs = fill_mask(masked_text)

            if outputs:
                for output in outputs:
                    if output['sequence'].strip() != text.strip():
                        new_word = output['token_str']
                        delta = len(new_word) - len(text_tokens[replace_index])
                        text = text[:word_start] + new_word + text[word_end:]
                        for ent in entities:
                            if ent['start'] > word_end:
                                ent['start'] += delta
                                ent['end'] += delta
                        break
        augmented_entities.extend([dict(ent) for ent in entities])

    return text, augmented_entities

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def apply_augmentation(row):
    augmented_text, augmented_entities = augment_text_and_entities(row['text'], row['entities'])
    row['text'] = augmented_text
    row['entities'] = augmented_entities
    return row

augmented_data = data.apply(apply_augmentation, axis=1)

In [ ]:
augmented_data.head(5)

,file_name,text,entities,sentence_id
0,172744.tsv,"нам прописали, так мой ребенок сыпью покрылся,...","[{'start': 123, 'entity_type': 'Drugform', 'en...",0
1,172744.tsv,Общее впечатление : я подошел\n,[],1
2,592814.tsv,Пила этот препарат для женщин 5 лет назад.\n,"[{'start': 23, 'entity_type': 'DI', 'end': 29,...",0
3,592814.tsv,"Так как начала работать на аптеке, начала част...",[],1
4,592814.tsv,В месяц по нескольку раз причем со всеми вытек...,"[{'start': 66, 'entity_type': 'DI', 'end': 72,...",2


In [ ]:
augmented_data.to_json('augmented_bert.json', orient='records', lines=True, force_ascii=False)

In [ ]:
with open(augmented_data_path, 'w', encoding='utf-8') as f_out:
    for line in tqdm(all_lines):
        json.dump(line, f_out, ensure_ascii=False)
        f_out.write('\n')

100%|██████████| 4809/4809 [00:00<00:00, 24681.34it/s]
